In [1]:
import yaml
import os
import datetime
import matplotlib.pyplot as plt
import arviz as az
import numpy as np
import pymc3 as pm
import pymc3.distributions.transforms as tr
import shutil
import theano
import theano.tensor as tt
import random
import math
import pandas as pd
from pymc3.math import *

print("Running on PyMC3 v{}".format(pm.__version__))

In /Users/tanavya/opt/anaconda3/envs/venv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/tanavya/opt/anaconda3/envs/venv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/tanavya/opt/anaconda3/envs/venv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/tanavya/opt/anaconda3/envs/venv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases lat

Running on PyMC3 v3.9.3


In [2]:
def get_situation(w, b):
    if 0 <= w and w <= 3:
        if 1 <= b and b <= 36:
            return 1
        elif 36 <= b and b <= 96:
            return 2
        else:
            return 3
    elif 4 <= w and w <= 6:
        if 1 <= b and b <= 36:
            return 4
        elif 36 <= b and b <= 96:
            return 5
        else:
            return 6
    else:
        if 1 <= b and b <= 36:
            return 7
        elif 36 <= b and b <= 96:
            return 8
        else:
            return 9

In [3]:
def get_outcome(r, wicket):
    if wicket is True:
        return 1
    elif r == 0:
        return 2
    elif r == 1:
        return 3
    elif r == 2:
        return 4
    elif r == 3:
        return 5
    elif r == 4:
        return 6
    elif r == 6:
        return 7
    else:
        return 0

In [4]:
data = pd.read_csv("data/deliveries.csv")

In [5]:
data.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [6]:
matches = pd.read_csv("data/matches.csv")

In [7]:
print("Number of matches in dataset:", len(matches))

Number of matches in dataset: 756


In [8]:
selected_ids = matches[matches["season"] >= 2015]["id"]
selected_data = data[data["match_id"].isin(selected_ids)]

In [9]:
selected_data.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,0,0,0,0,0,2,2,NaN,NaN,NaN


In [10]:
noballs_and_wides_count = 0
total_balls = 0
noballs_and_wides = {}
batsmen = []
bowlers = []
situation_data = {}

In [11]:
first_innings = selected_data[selected_data["inning"] == 1]
print("First innings data size:", len(first_innings))

First innings data size: 36516


In [12]:
batsman_index = {}
bowler_index = {}
batsmen = first_innings["batsman"].unique()
bowlers = first_innings["bowler"].unique()
for i in range(len(batsmen)):
    batsman_index[batsmen[i]] = i
for i in range(len(bowlers)):
    bowler_index[bowlers[i]] = i

X = [[] for i in range(9)]
id1 = [[] for i in range(9)]
id2 = [[] for i in range(9)]

for k in range(0, 8):
    noballs_and_wides[k] = 0
for l in range(0, 10):
    situation_data[l] = []

In [13]:
I = len(batsmen)
J = len(bowlers)

In [14]:
current_id = -1
w = 0
b = 0
for i in range(len(first_innings)):
    ball_data = first_innings.iloc[i]
    if current_id != ball_data["match_id"]:
        current_id = ball_data["match_id"]
        w = 0
        b = 0
        
    total_balls += 1
    b += 1
    
    batsman = ball_data["batsman"]
    bowler = ball_data["bowler"]
    
    player_dismissed = False
    if pd.notnull(first_innings["player_dismissed"].iloc[i]):
        player_dismissed = True
        w += 1
        
    if ball_data["wide_runs"] >= 1 or ball_data["noball_runs"] >= 1:
        noballs_and_wides_count += 1
        runs = ball_data["batsman_runs"] # check should be total_runs ?
        noballs_and_wides[get_outcome(runs, player_dismissed)] += 1
        continue
        
    runs = ball_data["batsman_runs"]
    l = get_situation(w, b) - 1
    k = get_outcome(runs, player_dismissed)
    if k == 0:
        continue
    X[l].append(k)
    id1[l].append(batsman_index[batsman])
    id2[l].append(bowler_index[bowler])
    situation_data[l].append((batsman_index[batsman], bowler_index[bowler], k))

In [15]:
noballs_and_wides_count

1225

In [16]:
X = np.asarray([np.array(X[i]) for i in range(9)], dtype=object)
id1 = np.asarray([np.array(id1[i]) for i in range(9)], dtype=object)
id2 = np.asarray([np.array(id2[i]) for i in range(9)], dtype=object)

In [17]:
for i in range(9):
    print("Balls in situation %d: %d" % (i+1, len(X[i])))

Balls in situation 1: 10329
Balls in situation 2: 13629
Balls in situation 3: 1949
Balls in situation 4: 47
Balls in situation 5: 3434
Balls in situation 6: 4497
Balls in situation 7: 2
Balls in situation 8: 181
Balls in situation 9: 1213


In [18]:
INF = 5
testval = [[-INF + x * (2 * INF)/5.0 for x in range(6)] for i in range(0, 9)]
l = [i for i in range(9)]

In [19]:
model = pm.Model()
Print = tt.printing.Print("shape:")
with model:
    delta_1 = pm.Uniform("delta_1", lower=0, upper=1)
    delta_2 = pm.Uniform("delta_2", lower=0, upper=1)
    inv_sigma_sqr = pm.Gamma("sigma^-2", alpha=1.0, beta=1.0)
    inv_tau_sqr = pm.Gamma("tau^-2", alpha=1.0, beta=1.0)
    mu_1 = pm.Normal("mu_1", mu=0, sigma=1/pm.math.sqrt(inv_tau_sqr), shape=len(batsmen))
    mu_2 = pm.Normal("mu_2", mu=0, sigma=1/pm.math.sqrt(inv_tau_sqr), shape=len(bowlers))
    delta = pm.math.ge(l, 3) * delta_1 + pm.math.ge(l, 6) * delta_2
    Print(delta.shape)
    Print(mu_1.shape)
    eta = [pm.Deterministic("eta_" + str(i), mu_1[id1[i]] - mu_2[id2[i]]) for i in range(9)]
    cutpoints = pm.Normal("cutpoints", mu=0, sigma=1/pm.math.sqrt(inv_sigma_sqr), transform=pm.distributions.transforms.ordered, shape=(9,6), testval=testval)
    Print(cutpoints.shape)
    X_ = [pm.OrderedLogistic("X_" + str(i), cutpoints=cutpoints[i], eta=eta[i], observed=X[i]-1) for i in range(9)]

shape: __str__ = [9]
shape: __str__ = [232]
shape: __str__ = [9 6]


In [ ]:
with model:
    trace = pm.sample(1000)

In [ ]:
#loc = pm.save_trace(trace)
loc = ".pymc_1.trace"

In [ ]:
with model:
    trace = pm.load_trace(loc)

In [ ]:
az.plot_trace(trace);

In [ ]:
az.summary(trace, round_to=5)

In [ ]:
pm.save_trace(trace=trace)

In [ ]:
cutpoints = np.mean(trace.get_values("cutpoints", burn=500, combine=True), axis=0)

In [ ]:
cutpoints

In [ ]:
mu_1 = np.mean(trace.get_values("mu_1", burn=500, combine=True), axis=0)
mu_2 = np.mean(trace.get_values("mu_2", burn=500, combine=True), axis=0)

In [ ]:
mu_1_sorted = sorted([(mu_1[i], batsmen[i]) for i in range(len(mu_1))])
mu_2_sorted = sorted([(mu_2[i], bowlers[i]) for i in range(len(mu_2))])

In [ ]:
mu_2_sorted

In [ ]:
print(mu_1.shape)
print(mu_2.shape)

In [ ]:
l = [i for i in range(9)]
delta_1 = np.mean(trace.get_values("delta_1", burn=500, combine=True), axis=0)
delta_2 = np.mean(trace.get_values("delta_1", burn=500, combine=True), axis=0)
delta = np.greater_equal(l, 3) * delta_1 + np.greater_equal(l, 6) * delta_2

In [ ]:
delta

In [ ]:
p = np.zeros(shape = (len(batsmen),len(bowlers),9,7))
for i in range(len(batsmen)):
    for j in range(len(bowlers)):
        for l in range(9):
            for k in range(7):
                if k == 0:
                    p[i,j,l,k] = 1/(1 + np.exp(-(cutpoints[l,k] - mu_1[i] + mu_2[j] - delta[l])))
                elif k == 6:
                    p[i,j,l,k] = 1 - 1/(1 + np.exp(-(cutpoints[l,k-1] - mu_1[i] + mu_2[j] - delta[l])))
                else:
                    p[i,j,l,k] = 1/(1 + np.exp(-(cutpoints[l,k] - mu_1[i] + mu_2[j] - delta[l]))) - 1/(1 + np.exp(-(cutpoints[l,k-1] - mu_1[i] + mu_2[j] - delta[l])))
                    
# can be vectorised

In [ ]:
pw = np.zeros(shape=7)          
for i in range(7):
    pw[i] =  float(noballs_and_wides[i])/noballs_and_wides_count
v = float(noballs_and_wides_count)/total_balls

In [ ]:
# Chosen match: Sunrisers Hyderabad vs Royal Challengers Bangalore 2017
batting_team = "Sunrisers Hyderabad"
bowling_team = "Royal Challengers Bangalore"
selected_data[(selected_data["match_id"] == 1) & (selected_data["inning"] == 1)]["batsman"].unique()

In [ ]:
for batsman in batsmen:
    if "Nehra" in batsman:
        print(batsman)

In [ ]:
batting_order_names = ['DA Warner', 'S Dhawan', 'MC Henriques', 'Yuvraj Singh',
       'DJ Hooda', 'BCJ Cutting', 'NV Ojha', 'B Kumar', 'Bipul Sharma', 'Rashid Khan', 'A Nehra']
batting_order = []
for batsman in batting_order_names:
    batting_order.append(batsman_index[batsman])

In [ ]:
batting_order

In [ ]:
current_bowler = ""
bowling_order = []
bowling_order_names = []
for bowler in selected_data[(selected_data["match_id"] == 1) & (selected_data["inning"] == 1)]["bowler"]:
    if bowler != current_bowler:
        current_bowler = bowler
        bowling_order_names.append(bowler)
        bowling_order.append(bowler_index[bowler])

In [ ]:
bowling_order_names

In [ ]:
bowling_order

In [ ]:
selected_data.columns

In [ ]:
columns=["inning", "batting_team", "bowling_team", "over", "ball", "batsman", "non_striker", "bowler", "wide_runs", "batsman_runs", "total_runs", "player_dismissed"]

In [ ]:
first_innings = []

In [ ]:
X1 = [-1 for i in range(120)]
Y1 = [0 for i in range(120)]
q1 = [0 for i in range(120)]
wickets = 0
runs = 0

In [ ]:
q1[0] = (batting_order[0], batting_order[1])

In [ ]:
# FIRST INNINGS SIMULATION

for b in range(120):
    
    if wickets == 10:
        X1[b] = -1
        
    else:
        wide_runs = 0 
        batsman_runs = 0
        total_runs = 0
        player_dismissed = None
        
        j = bowling_order[int(b/6)]
        
        bowler = bowlers[j]
        inning = 1
        over = int(b/6)
        ball = (b%6) + 1

        while np.random.uniform(0, 1) < v:       
            random = np.random.uniform(0, 1)
            Y1[b] = (random > pw[0]) + (random > pw[0] + pw[1]) + (random > pw[0] + pw[1] + pw[2]) + \
                    (random > pw[0] + pw[1] + pw[2] + pw[3]) + (random > pw[0] + pw[1] + pw[2] + pw[3] + pw[4]) + \
                    (random > pw[0] + pw[1] + pw[2] + pw[3] + pw[4] + pw[5])
            # should fix Y1[b] replaces previous Y1[b]
            runs += 1
            
            wide_runs = 1            
            
            if Y1[b] == 0:
                wickets += 1 # fix wicket fallen on wide/noball
            elif Y1[b] == 1:
    
                runs += 0
            elif Y1[b] == 2:
                wide_runs = 1
                runs += 1
            elif Y1[b] == 3:
                wide_runs = 2
                runs += 2
            elif Y1[b] == 4:
                wide_runs = 3
                runs += 3
            elif Y1[b] == 5:
                wide_runs = 4
                runs += 4
            elif Y1[b] == 6:
                wide_runs = 5
                runs += 6  
            
            batsman = batsmen[q1[b-1][0]]
            non_striker = batsmen[q1[b-1][1]]
        
            first_innings.append([inning, batting_team, bowling_team, over, ball, batsman, non_striker, bowler, wide_runs, batsman_runs, total_runs, player_dismissed])

        #batsman to face the delivered ball
        
        if b % 6 == 0:                                              # First ball of any over
            if X1[b-1] == 1:                                        # Wicket on last ball
                player_dismissed = batsmen[q1[b-1][0]]
                q1[b] = (q1[b-1][1], batting_order[wickets + 1])    # non striker now on strike
            elif X1[b-1] == 2 | X1[b-1] == 4:                       # Batsmen rotated places while running btw wickets on last ball
                q1[b] = q1[b-1]                                     # single or triple on last ball so same striker
            else:
                q1[b] = (q1[b-1][1], q1[b-1][0])

        else:    
            if X1[b-1] == 1:                                        # Wicket on last ball
                player_dismissed = batsmen[q1[b-1][0]]
                q1[b] = (batting_order[wickets + 1], q1[b-1][1])    # new batsman on strike
            elif X1[b-1] == 3 | X1[b-1] == 5:                       # Batsmen rotated places while running btw wickets on last ball
                q1[b] = (q1[b-1][1], q1[b-1][0])
            else:
                q1[b] = q1[b-1]
        

        batsman = batsmen[q1[b][0]]
        non_striker = batsmen[q1[b][1]]
        first_innings.append([inning, batting_team, bowling_team, over, ball, batsman, non_striker, bowler, wide_runs, batsman_runs, total_runs, player_dismissed])
        
        #bowler to ball 
        
        l = get_situation(wickets, b) - 1
        rand = np.random.uniform(0, 1)
        
        # following can be simplified
        q = q1[b][0]
        X1[b] = 0 + (rand > p[q,j,l,0]) + (rand > (p[q,j,l,0]+p[q,j,l,1])) + (rand > (p[q,j,l,0]+p[q,j,l,1]+p[q,j,l,2])) + (rand > (p[q,j,l,0]+p[q,j,l,1]+p[q,j,l,2]+p[q,j,l,3])) + (rand > (p[q,j,l,0]+p[q,j,l,1]+p[q,j,l,2]+p[q,j,l,3]+p[q,j,l,4])) + (rand > (p[q,j,l,0]+p[q,j,l,1]+p[q,j,l,2]+p[q,j,l,3]+p[q,j,l,4]+p[q,j,l,5]))
        print(b, X1[b])
        if X1[b] == 2:
            runs += 1
        elif X1[b] == 3:
            runs += 2
        
        elif X1[b] == 4:
            runs += 3
            
        elif X1[b] == 5 :
            runs += 4
        
        elif X1[b] == 6:
            runs += 6
            
        elif X1[b] == 0:
            wickets += 1

In [ ]:
first_innings_df = pd.DataFrame(first_innings, columns = columns)

In [ ]:
first_innings_df

In [47]:
print("Total Runs:", runs)
print("Wickets fallen:", wickets)

Total Runs: 4


NameError: name 'wickets' is not defined

In [51]:
DLS = pd.read_csv("data/dls-simplified.csv").rename(columns={"Unnamed: 0": "Balls Left"}).set_index("Balls Left")
DLS.columns = DLS.columns.astype(int)
DLS.index = DLS.index.astype(int)

In [65]:
DLS

,10,9,8,7,6,5,4,3,2,1,0
Balls Left,,,,,,,,,,,
120,1.000000,0.969000,0.930000,0.879000,0.813000,0.722000,0.599000,0.448000,0.297000,0.176000,0
119,0.992667,0.959000,0.921167,0.870833,0.805667,0.715500,0.593333,0.443333,0.292833,0.172167,0
118,0.985333,0.949000,0.912333,0.862667,0.798333,0.709000,0.587667,0.438667,0.288667,0.168333,0
117,0.978000,0.939000,0.903500,0.854500,0.791000,0.702500,0.582000,0.434000,0.284500,0.164500,0
116,0.970667,0.929000,0.894667,0.846333,0.783667,0.696000,0.576333,0.429333,0.280333,0.160667,0
...,...,...,...,...,...,...,...,...,...,...,...
4,0.091333,0.075333,0.064667,0.056667,0.048667,0.044667,0.040000,0.034667,0.028000,0.006000,0
3,0.068500,0.056500,0.048500,0.042500,0.036500,0.033500,0.030000,0.026000,0.021000,0.004500,0
2,0.045667,0.037667,0.032333,0.028333,0.024333,0.022333,0.020000,0.017333,0.014000,0.003000,0


In [130]:
#Resources lost due to wicket
y = np.zeros(shape=(121, 10))
for i in range(10):
    y[:,i] = DLS.loc[:,(i+1)] - DLS.loc[:,i]
y = pd.DataFrame(y).iloc[:, ::-1]
y.columns = [i for i in range(10)]

In [131]:
y

,0,1,2,3,4,5,6,7,8,9
0,0.031000,0.039000,0.051000,0.066000,0.091000,0.123000,0.151000,0.151000,0.121000,0.176000
1,0.033667,0.037833,0.050333,0.065167,0.090167,0.122167,0.150000,0.150500,0.120667,0.172167
2,0.036333,0.036667,0.049667,0.064333,0.089333,0.121333,0.149000,0.150000,0.120333,0.168333
3,0.039000,0.035500,0.049000,0.063500,0.088500,0.120500,0.148000,0.149500,0.120000,0.164500
4,0.041667,0.034333,0.048333,0.062667,0.087667,0.119667,0.147000,0.149000,0.119667,0.160667
...,...,...,...,...,...,...,...,...,...,...
116,0.016000,0.010667,0.008000,0.008000,0.004000,0.004667,0.005333,0.006667,0.022000,0.006000
117,0.012000,0.008000,0.006000,0.006000,0.003000,0.003500,0.004000,0.005000,0.016500,0.004500
118,0.008000,0.005333,0.004000,0.004000,0.002000,0.002333,0.002667,0.003333,0.011000,0.003000
119,0.004000,0.002667,0.002000,0.002000,0.001000,0.001167,0.001333,0.001667,0.005500,0.001500


In [132]:
x = np.zeros(shape = (120, 11))
for i in range(120):
    x[i,:] = DLS.loc[i+1,:] - DLS.loc[i,:]
x = pd.DataFrame(x).iloc[::-1, :]
x.index = x.index[::-1]

In [133]:
x

,0,1,2,3,4,5,6,7,8,9,10
0,0.007333,0.010000,0.008833,0.008167,0.007333,0.006500,0.005667,0.004667,0.004167,0.003833,0.0
1,0.007333,0.010000,0.008833,0.008167,0.007333,0.006500,0.005667,0.004667,0.004167,0.003833,0.0
2,0.007333,0.010000,0.008833,0.008167,0.007333,0.006500,0.005667,0.004667,0.004167,0.003833,0.0
3,0.007333,0.010000,0.008833,0.008167,0.007333,0.006500,0.005667,0.004667,0.004167,0.003833,0.0
4,0.007333,0.010000,0.008833,0.008167,0.007333,0.006500,0.005667,0.004667,0.004167,0.003833,0.0
...,...,...,...,...,...,...,...,...,...,...,...
115,0.022833,0.018833,0.016167,0.014167,0.012167,0.011167,0.010000,0.008667,0.007000,0.001500,0.0
116,0.022833,0.018833,0.016167,0.014167,0.012167,0.011167,0.010000,0.008667,0.007000,0.001500,0.0
117,0.022833,0.018833,0.016167,0.014167,0.012167,0.011167,0.010000,0.008667,0.007000,0.001500,0.0
118,0.022833,0.018833,0.016167,0.014167,0.012167,0.011167,0.010000,0.008667,0.007000,0.001500,0.0


In [ ]:
target = runs

In [ ]:
X2 = [-1 for i in range(120)]
Y2 = [0 for i in range(120)]
q2 = [0 for i in range(120)]
wickets2 = 0
runs2 = 0
second_innings = []

In [ ]:
q2[0] = (batting_order[0], batting_order[1])

In [ ]:
# SECOND INNINGS SIMULATION

for b in range(120):
    
    if wickets2 == 10:
        X2[b] = -1
        
    else:
        wide_runs = 0 
        batsman_runs = 0
        total_runs = 0
        player_dismissed = None
        
        j = bowling_order[int(b/6)]
        
        bowler = bowlers[j]
        inning = 1
        over = int(b/6)
        ball = (b%6) + 1

        while np.random.uniform(0, 1) < v:       
            random = np.random.uniform(0, 1)
            Y2[b] = (random > pw[0]) + (random > pw[0] + pw[1]) + (random > pw[0] + pw[1] + pw[2]) + \
                    (random > pw[0] + pw[1] + pw[2] + pw[3]) + (random > pw[0] + pw[1] + pw[2] + pw[3] + pw[4]) + \
                    (random > pw[0] + pw[1] + pw[2] + pw[3] + pw[4] + pw[5])
            # should fix Y2[b] replaces previous Y2[b]
            runs2 += 1
            
            wide_runs = 1            
            
            if Y2[b] == 0:
                wickets2 += 1 # fix wicket fallen on wide/noball
                
            elif Y2[b] == 1:
                runs2 += 0
                
            elif Y2[b] == 2:
                wide_runs = 1
                runs2 += 1
                
            elif Y2[b] == 3:
                wide_runs = 2
                runs2 += 2
                
            elif Y2[b] == 4:
                wide_runs = 3
                runs2 += 3
                
            elif Y2[b] == 5:
                wide_runs = 4
                runs2 += 4
                
            elif Y2[b] == 6:
                wide_runs = 5
                runs2 += 6  
            
            batsman = batsmen[q2[b-1][0]]
            non_striker = batsmen[q2[b-1][1]]
        
            second_innings.append([inning, batting_team, bowling_team, over, ball, batsman, non_striker, bowler, wide_runs, batsman_runs, total_runs, player_dismissed])

        #batsman to face the delivered ball
        
        if b % 6 == 0:                                              # First ball of any over
            if X2[b-1] == 1:                                        # Wicket on last ball
                player_dismissed = batsmen[q2[b-1][0]]
                q2[b] = (q2[b-1][1], batting_order[wickets2 + 1])    # non striker now on strike
            elif X2[b-1] == 2 | X2[b-1] == 4:                       # Batsmen rotated places while running btw wickets2 on last ball
                q2[b] = q2[b-1]                                     # single or triple on last ball so same striker
            else:
                q2[b] = (q2[b-1][1], q2[b-1][0])

        else:    
            if X2[b-1] == 1:                                        # Wicket on last ball
                player_dismissed = batsmen[q2[b-1][0]]
                q2[b] = (batting_order[wickets2 + 1], q2[b-1][1])    # new batsman on strike
            elif X2[b-1] == 3 | X2[b-1] == 5:                       # Batsmen rotated places while running btw wickets2 on last ball
                q2[b] = (q2[b-1][1], q2[b-1][0])
            else:
                q2[b] = q2[b-1]


        batsman = batsmen[q2[b][0]]
        non_striker = batsmen[q2[b][1]]
        second_innings.append([inning, batting_team, bowling_team, over, ball, batsman, non_striker, bowler, wide_runs, batsman_runs, total_runs, player_dismissed])
        
        
        #estimation of probability of outcome in 2nd inning
        
        
        p2 = np.zeros(7)
        
        l = get_situation(wickets2, b) - 1
        q = q2[b][0]
        
        E1 = p[q,j,l,2] + 2 * p[q,j,l,3] + 3 * p[q,j,l,4] + 4 * p[q,j,l,5] + 6 * p[q,j,l,6]
        E2 = x[b, wickets2] + y[b, wickets2] * p[q,j,l,1]
        
        d = E2/(E2 + y[b, wickets2] * (1 - p[q,j,l,1] - p[q,j,l,2]))
        
        c = DLS[b, wickets2] * E1 / ((target - runs2 + 1) * E2)
        
        p2[1] = c * p[q,j,l,2]
        p2[0] = p[q,j,l,1] + delta* p[q,j,l,2] * (1-c)
        
        for k in range(2,7):
            p2[k] = ((1 - p[q,j,l,0] - (c + d * (1-c)) * p[q,j,l,1]) / (1 - p[q,j,l,0] - p[q,j,l,1])) * p[q,j,l,k]
        
        random = np.random.uniform(0, 1)
        X2[b] = 0 + (random > p2[0]) + (random > (p2[0]+p2[1])) + (random > (p2[0]+p2[1]+p2[2])) + (random > (p2[0]+p2[1]+p2[2]+p2[3])) + (random > (p2[0]+p2[1]+p2[2]+p2[3]+p2[4])) + (random > (p2[0]+p2[1]+p2[2]+p2[3]+p2[4]+p2[5]))
        
        print(b, X2[b])
        
        if X2[b] == 2:
            runs2 += 1
            
        elif X2[b] == 3:
            runs2 += 2
        
        elif X2[b] == 4:
            runs2 += 3
            
        elif X2[b] == 5 :
            runs2 += 4
        
        elif X2[b] == 6:
            runs2 += 6
            
        elif X2[b] == 0:
            wickets2 += 1